In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter
import torch.utils.checkpoint as checkpoint

import matplotlib.pyplot as plt

from torchvision.io import read_image

In [2]:
#Kernel size k x k x k
# 1. Depthwise Convolution with kernel size k × k × k
# 2. Normalization, with C output channels
#       We use channel-wise GroupNorm for stability with small
#       batches, instead of the original LayerNorm
class DepthwiseConv3d(nn.Module):
    def __init__(self, in_channels, kernel_size, padding=0, bias=False):
        super(DepthwiseConv3d, self).__init__()
        self.depthwise_conv = nn.Conv3d(in_channels, in_channels, 
                                        kernel_size=kernel_size, padding=padding, 
                                        groups=in_channels, bias=bias)
        self.norm = nn.GroupNorm(num_groups=in_channels, num_channels=in_channels)

    def forward(self, x):
        out = self.depthwise_conv(x)
        out = self.norm(out)
        return out


# Expansion layer contains:
# 1. An overcomplete Convolution Layer with CR output channels,
#       where R is the expansion ratio (expansion_ratio)
# 

class ExpansionLayer(nn.Module):
    def __init__(self, in_channels, expansion_ratio, kernel_size, stride, padding):
        super(ExpansionLayer, self).__init__()
        self.conv = nn.Conv3d(in_channels, in_channels * expansion_ratio, 
                              kernel_size, stride=stride, padding=padding, 
                              groups=in_channels)
        self.norm = nn.GroupNorm(in_channels, in_channels)
        self.activation = nn.GELU()

    def forward(self, x):
        out = self.conv(x)
        out = self.norm(out)
        out = self.activation(out)
        return out
    
# Compression layer:
# 1. 1×1×1 kernel and and C output channels performing channel-wise 
#       compression of the feature maps.
# 2. It can have 2xC or 0.5xC
class CompressionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CompressionLayer, self).__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [3]:
class ResidualLayer(nn.Module):
    def __init__(self, in_channels, kernel_size, expansion_ratio, out_channels, padding):
        super(ResidualLayer, self).__init__()
        stride = 2
        if in_channels == out_channels: stride = 1
        self.sampling_ratio = out_channels/in_channels
        
        self.depthwise_conv = DepthwiseConv3d(in_channels, kernel_size, padding)
        self.expansion_layer = ExpansionLayer(in_channels, expansion_ratio, 1 , stride, padding)
        self.compression_layer = CompressionLayer(in_channels * expansion_ratio, out_channels)
        
        # Layer only for sampling ratio not 1 
        self.conv_transpose = nn.Conv3d(in_channels, in_channels * self.sampling_ratio, 
                              kernel_size = 1, stride=2, groups=in_channels)

        # self.block_expansion_ratio = out_channels/in_channels # It can have 2xC or 0.5xC

    def forward(self, x):
        out = self.depthwise_conv(x)
        out = self.expansion_layer(out)
        out = self.compression_layer(out)
        if self.sampling_ratio != 1:
            return out + self.conv_transpose(x)
        return out + x #Residual block (is it just sum)

In [ ]:
# kernel size 5x5x5 - most effective
# Expansion Ratios: 
# R1 = R9 = 3
# R2 = R8 = 4
# R3-R7 = 8

# Number of blocks
# B1 = B9 = 3
# B2 = B8 = 4
# B3−7 = 8 

#stride = 1 if in == out, and 2 if in != out

# MedNeXt-B (5 × 5 × 5) + UpKern 84.23 87.06 89.38 92.36

kernel_size = 5

#B1 and B9 
class MedNeXtBlock_x3(nn.Module):
    def __init__(self, in_channels):
        super(MedNeXtBlock_x3, self).__init__()
        R1 = 3 # Expansion Ratio 
        self.l1 = ResidualLayer(in_channels, kernel_size, R1, in_channels, (2, 2, 2))
        self.l2 = ResidualLayer(in_channels, kernel_size, R1, in_channels, (2, 2, 2))
        self.l3 = ResidualLayer(in_channels, kernel_size, R1, in_channels, (2, 2, 2))

    def forward(self, x):
        out = self.l1 (x)
        out = self.l2 (out)
        out = self.l3 (out)
        return out
        
#B2 and B8 
class MedNeXtBlock_x4(nn.Module):
    def __init__(self, in_channels):
        super(MedNeXtBlock_x4, self).__init__()
        R2 = 4 # Expansion Ratio 
        self.l1 = ResidualLayer(in_channels, kernel_size, R2, in_channels, (2, 2, 2))
        self.l2 = ResidualLayer(in_channels, kernel_size, R2, in_channels, (2, 2, 2))
        self.l3 = ResidualLayer(in_channels, kernel_size, R2, in_channels, (2, 2, 2))
        self.l4 = ResidualLayer(in_channels, kernel_size, R2, in_channels, (2, 2, 2))

    def forward(self, x):
        out = self.l1 (x)
        out = self.l2 (out)
        out = self.l3 (out)
        out = self.l4 (out)
        return out
    
#B3-B7
class MedNeXtBlock_x8(nn.Module):
    def __init__(self, in_channels):
        super(MedNeXtBlock_x8, self).__init__()
        R3 = 8 # Expansion Ratio 
        self.l1 = ResidualLayer(in_channels, kernel_size, R3, in_channels, (2, 2, 2))
        self.l2 = ResidualLayer(in_channels, kernel_size, R3, in_channels, (2, 2, 2))
        self.l3 = ResidualLayer(in_channels, kernel_size, R3, in_channels, (2, 2, 2))
        self.l4 = ResidualLayer(in_channels, kernel_size, R3, in_channels, (2, 2, 2))
        self.l5 = ResidualLayer(in_channels, kernel_size, R3, in_channels, (2, 2, 2))
        self.l6 = ResidualLayer(in_channels, kernel_size, R3, in_channels, (2, 2, 2))
        self.l7 = ResidualLayer(in_channels, kernel_size, R3, in_channels, (2, 2, 2))
        self.l8 = ResidualLayer(in_channels, kernel_size, R3, in_channels, (2, 2, 2))

    def forward(self, x):
        out = self.l1 (x)
        out = self.l2 (out)
        out = self.l3 (out)
        out = self.l4 (out)
        out = self.l5 (out)
        out = self.l6 (out)
        out = self.l7 (out)
        out = self.l8 (out)
        return out
    

In [4]:
# Expansion Ratios: 
# R1 = R9 = 3
# R2 = R8 = 4
# R3-R7 = 8

# Number of blocks
# B1 = B9 = 3
# B2 = B8 = 4
# B3−7 = 8 

class MedNeXt(nn.Module):
    def __init__(self, in_channels):
        super(MedNeXt, self).__init__()
        # Expansion Ratios
        R1 = 3 # R9
        R2 = 4 # R8
        R3 = 8 # R3-R7
        C = in_channels

        # Encoder
        self.l1 = MedNeXtBlock_x3 (C)
        self.l2 = ResidualLayer(C, kernel_size, R2, C*2, (2, 2, 2)) # Down 2x
        self.l3 = MedNeXtBlock_x4 (C*2)
        self.l4 = ResidualLayer(C*2, kernel_size, R3, C*4, (2, 2, 2)) # Down 2x
        self.l5 = MedNeXtBlock_x8 (C*4)
        self.l6 = ResidualLayer(C*4, kernel_size, R3, C*8, (2, 2, 2)) # Down 2x
        self.l7 = MedNeXtBlock_x8 (C*8)
        self.l8 = ResidualLayer(C*8, kernel_size, R3, C*16, (2, 2, 2)) # Down 2x

        # Bottleneck
        self.bottleneck = ResidualLayer(C*16, kernel_size, R3, C*16, (2, 2, 2))

        # Decoder
        self.l9 = ResidualLayer(C*16, kernel_size, R3, C*8, (2, 2, 2)) # Up 2x
        self.l10 = MedNeXtBlock_x8 (C*8)
        self.l11 = ResidualLayer(C*8, kernel_size, R3, C*4, (2, 2, 2)) # Up 2x
        self.l12 = MedNeXtBlock_x8 (C*4)
        self.l11 = ResidualLayer(C*4, kernel_size, R3, C*2, (2, 2, 2)) # Up 2x
        self.l12 = MedNeXtBlock_x4 (C*2)
        self.l11 = ResidualLayer(C*2, kernel_size, R2, C*1, (2, 2, 2)) # Up 2x
        self.l12 = MedNeXtBlock_x3 (C)

    def forward(self, x):


IndentationError: expected an indented block (Temp/ipykernel_11620/1260952322.py, line 15)

In [10]:
# Creating a dataset 

# read a JPEG image
img = read_image('.\\Datasets\\chest_xray\\train\\NORMAL\\IM-0115-0001.jpeg')

# display the image properties
print("Image data:", img)

# check if input image is a PyTorch tensor
print("Is image a PyTorch Tensor:", torch.is_tensor(img))
print("Type of Image:", type(img))

# size of the image
print(img.size())

# convert the torch tensor to PIL image
img = transforms.ToPILImage()(img)

# display the image
img.show()


Image data: tensor([[[ 22,  24,  24,  ...,  96,  94,  93],
         [ 23,  24,  24,  ...,  97,  94,  92],
         [ 24,  24,  23,  ...,  99,  96,  94],
         ...,
         [ 49,  50,  48,  ..., 119, 119, 119],
         [ 50,  52,  53,  ..., 123, 124, 123],
         [ 52,  53,  54,  ..., 129, 128, 127]]], dtype=torch.uint8)
Is image a PyTorch Tensor: True
Type of Image: <class 'torch.Tensor'>
torch.Size([1, 1858, 2090])


In [ ]:
def gelu(x):
    """Implementation of the gelu activation function."""
    return x * torch.sigmoid(1.702 * x)

In [2]:
class DepthwiseConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(DepthwiseConv2d, self).__init__()
        self.depthwise_conv = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride,
                                        padding=padding, groups=in_channels)
        self.bn = nn.BatchNorm2d(in_channels)
        self.pointwise_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pointwise_conv(x)
        x = self.relu(x)
        return x


In [4]:
train_dataset = datasets.CIFAR10(root='./data', train=True, 
                                 download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, 
                                download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified
